In [19]:
import pandas as pd

import os
import datetime

In [20]:
wkdir = "/Users/chrisolen/Documents/uchicago_courses/deep_learning_and_image_recognition/finance/"
data_files = os.listdir(wkdir+'data')
data_files.remove('.DS_Store')

In [21]:
# Loading time series from file:

df_list = []

for i in range(len(data_files)):
    temp_df = pd.read_csv(wkdir+'data/'+ data_files[i], encoding = 'unicode_escape', header=None ).iloc[6:,]
    df_list.append(temp_df)
    
    
    

In [22]:
# Cleaning and merging time series:

assets = df_list[0]
assets.rename({0:'date'}, axis=1, inplace=True)
etf_idx = []
gdps = pd.DataFrame()
unemp = pd.DataFrame()

counter_daily = 0
counter_quarterly = 0
counter_monthly = 0


for i in range(1,len(df_list)):
    
    if len(df_list[i][0].iloc[0]) < 8: # reformatting if the date string is smaller than m/d/yyyy
        df_list[i][0]=[datetime.datetime.strptime(df_list[i][0].iloc[j], "%m/%d/%y").strftime('%-m/%-d/%Y') for j in range(len(df_list[i]))]
    else:
        pass

    if len(df_list[i])==5218: # for daily time series
        counter_daily += 1
        assets = assets.merge(df_list[i], left_on=assets['date'], right_on=df_list[i][0])
        assets.drop(['key_0',0], axis=1, inplace=True)
        assets.rename(columns={assets.columns[1+counter_daily]:data_files[i][:-4]}, inplace=True)
        print("iteration {} is done".format(i))
        
    elif len(df_list[i])==81: # for quarterly time series
        
        if gdps.shape[0]==0: # keeping both date and values for the first one
            counter_quarterly += 1
            df_list[i].rename({0:'date'}, axis=1, inplace=True)
            gdps = pd.concat([gdps,df_list[i]], axis=1)
            gdps.rename(columns={gdps.columns[counter_quarterly]:data_files[i][:-4]}, inplace=True)
            print("iteration {} is done".format(i))
            
        else: # keeping only values for the rest
            counter_quarterly += 1
            gdps = pd.concat([gdps,df_list[i].iloc[:,1]], axis=1)
            gdps.rename(columns={gdps.columns[counter_quarterly]:data_files[i][:-4]}, inplace=True)
            print("iteration {} is done".format(i))
            
    else: # for monthly time series
        counter_monthly += 1
        df_list[i].rename({0:'date'}, axis=1, inplace=True)
        unemp = pd.concat([unemp,df_list[i]], axis=1)
        unemp.rename(columns={unemp.columns[counter_monthly]:data_files[i][:-4]}, inplace=True)
        print("iteration {} is done".format(i))
        
assets.rename(columns={assets.columns[1]:data_files[0][:-4]}, inplace=True) # One more colunn to rename

# Resetting indexes with date

assets.set_index('date', drop = True, inplace=True)
gdps.set_index('date', drop = True, inplace=True)
unemp.set_index('date', drop = True, inplace=True)

# Joining dfs
first_join = assets.join(gdps, how = 'left')
features = first_join.join(unemp, how = 'left')
    

iteration 1 is done
iteration 2 is done
iteration 3 is done
iteration 4 is done
iteration 5 is done
iteration 6 is done
iteration 7 is done
iteration 8 is done
iteration 9 is done
iteration 10 is done
iteration 11 is done
iteration 12 is done
iteration 13 is done
iteration 14 is done
iteration 15 is done
iteration 16 is done
iteration 17 is done
iteration 18 is done
iteration 19 is done
iteration 20 is done
iteration 21 is done
iteration 22 is done
iteration 23 is done
iteration 24 is done
iteration 25 is done
iteration 26 is done


In [5]:
# Given data availability, we begin in 2008:

features = features.loc['1/2/2008':,:] 

In [6]:
features['CHNGDP'].isna().value_counts()


True     3037
False      33
Name: CHNGDP, dtype: int64

In [7]:
features.tail()

,UST_10YR,UIVE_SP500VALUEETF,VNQ_VANGREALEST,USFFR,EMB_USDEMRGBOND,LQD_CORPBOND,MUB_MUNIBOND,SHY_1-3USTR,USDJPY,USDGBP,...,XLE_ENERGYSPDR,SP500_GSCI,EFA_MSCIEAFE,TIP_TIPSBOND,UST_2YR,USDOIS,CHNGDP,USGDP,EZGDP,US_UNEMP
date,,,,,,,,,,,,,,,,,,,,,
10/2/2019,1.5969,116.89,91.98,1.85,112.39,127.24,114.13,84.87,0.933,1.23015,...,56.41,397.184,25.61,116.32,1.483,1.836,NaN,NaN,NaN,NaN
10/3/2019,1.5353,115.31,92.87,1.83,112.93,127.74,114.37,85.05,0.9353499,1.23295,...,57.02,397.356,25.73,116.69,1.3871,1.808,NaN,NaN,NaN,NaN
10/4/2019,1.5131,116.48,93.43,1.82,113.92,128.27,114.37,85.04,0.93505,1.23335,...,57.38,399.469,25.92,117.05,1.3947,1.798,NaN,NaN,NaN,NaN
10/7/2019,1.5523,117.46,93.23,1.82,113.28,127.62,114.28,84.95,0.932,1.2292,...,56.87,399.211,25.97,116.73,1.4668,1.777,NaN,NaN,NaN,NaN
10/8/2019,1.5386,116.35,92.74,1.82,113.17,127.51,114.61,85,0.93385,1.2219,...,55.9,399.49,25.75,116.62,1.4265,1.769,NaN,NaN,NaN,NaN


In [8]:
features.shape

(3070, 27)

In [9]:
# Converting everything from string to float:

for i in range(len(features.columns)):
    features.loc[:,features.columns[i]] = features.loc[:,features.columns[i]].astype(float)
    

In [10]:
features.dtypes

UST_10YR              float64
UIVE_SP500VALUEETF    float64
VNQ_VANGREALEST       float64
USFFR                 float64
EMB_USDEMRGBOND       float64
LQD_CORPBOND          float64
MUB_MUNIBOND          float64
SHY_1-3USTR           float64
USDJPY                float64
USDGBP                float64
VIG_VANGDIV           float64
IVV_SP500             float64
USDRMB                float64
CRUDOIL               float64
CFE_VIX               float64
EEM_MSCIEMERGING      float64
USDEUR                float64
XLE_ENERGYSPDR        float64
SP500_GSCI            float64
EFA_MSCIEAFE          float64
TIP_TIPSBOND          float64
UST_2YR               float64
USDOIS                float64
CHNGDP                float64
USGDP                 float64
EZGDP                 float64
US_UNEMP              float64
dtype: object

In [11]:
# Creating Macroeconomic 'Shock' Variables:

features[['CHNGDP_Shock','USGDP_Shock','EZGDP_Shock','US_UNEMP_Shock']] = features.loc[:,['CHNGDP','USGDP','EZGDP','US_UNEMP']].fillna(0)

In [12]:
# Filling in Nans from Macroeconomic Variables:

list_for_backfills = ['CHNGDP','USGDP','EZGDP','US_UNEMP']
for i in range(len(list_for_backfills)):
    features.loc[:,list_for_backfills[i]].fillna(method='ffill', inplace=True) # propagating last valid value before series of nans forward to fillna
    features.loc[:,list_for_backfills[i]].fillna(method='bfill', inplace=True) # propagating first valid value back to fillnas


In [13]:
features.head()

,UST_10YR,UIVE_SP500VALUEETF,VNQ_VANGREALEST,USFFR,EMB_USDEMRGBOND,LQD_CORPBOND,MUB_MUNIBOND,SHY_1-3USTR,USDJPY,USDGBP,...,UST_2YR,USDOIS,CHNGDP,USGDP,EZGDP,US_UNEMP,CHNGDP_Shock,USGDP_Shock,EZGDP_Shock,US_UNEMP_Shock
date,,,,,,,,,,,,,,,,,,,,,
1/2/2008,3.8971,76.39,61.27,4.11,101.490,106.08,102.28,82.3401,0.91129,1.98135,...,2.8658,4.139,85399.6,15671.383,2616.0,5.0,0.0,0.0,0.0,0.0
1/3/2008,3.9045,75.51,59.36,4.25,101.670,105.73,102.16,82.4700,0.91320,1.97257,...,2.8248,4.165,85399.6,15671.383,2616.0,5.0,0.0,0.0,0.0,0.0
1/4/2008,3.8542,74.50,56.95,4.18,101.720,106.60,102.11,82.5900,0.92056,1.97325,...,2.7336,4.137,85399.6,15671.383,2616.0,5.0,0.0,0.0,0.0,0.0
1/7/2008,3.8372,73.98,57.48,4.27,101.950,106.47,102.00,82.5400,0.91537,1.96882,...,2.7655,4.071,85399.6,15671.383,2616.0,5.0,0.0,0.0,0.0,0.0
1/8/2008,3.8389,74.28,55.49,4.27,102.197,106.27,101.85,82.6600,0.91798,1.97279,...,2.7607,4.059,85399.6,15671.383,2616.0,5.0,0.0,0.0,0.0,0.0


In [14]:
# Separating features into macroeconomic indictors and portfolio:

macroecon = ['UST_10YR','USFFR','USDRMB','CRUDOIL','CFE_VIX','USDEUR','UST_2YR',
             'SP500_GSCI','USDOIS','CHNGDP','USGDP','EZGDP','US_UNEMP','CHNGDP_Shock',
            'USGDP_Shock','EZGDP_Shock','US_UNEMP_Shock','USDJPY','USDGBP','UIVE_SP500VALUEETF']

portfolio = ['VNQ_VANGREALEST','EMB_USDEMRGBOND','LQD_CORPBOND',
            'MUB_MUNIBOND','VIG_VANGDIV','IVV_SP500','EEM_MSCIEMERGING',
            'EFA_MSCIEAFE','XLE_ENERGYSPDR','SHY_1-3USTR','TIP_TIPSBOND',]

In [15]:
# Creating volatility features for portfolio assets:

for i in range(len(portfolio)):
    features = features.join(features[portfolio].iloc[:,i].rolling(5).std(), how = 'left', rsuffix = '_')
    features.rename(columns={features.columns[-1:][0]:portfolio[i][0:3]+"_VOL"}, inplace = True)
    



In [16]:
features.shape

(3070, 42)

In [17]:
features.dropna(inplace=True)

In [18]:
features.shape

(2959, 42)